#Import Libraries

In [ ]:
import pandas as pd
from statsmodels.stats.proportion import proportions_ztest
import statsmodels.api as sm
import numpy as np
from datetime import datetime, timedelta
from statsmodels.stats.power import NormalIndPower

# Generate mock A/B test data

In [ ]:
np.random.seed(42)
num_users_per_group = 5000
start_date = datetime(2023, 11, 1)
test_duration_days = 14 # 2 week test

Control Group (A)

In [ ]:
control_user_ids = [f'UserA_{10000+i}' for i in range(num_users_per_group)]
control_group_assignment = ['Control'] * num_users_per_group

Assume a baseline conversion rate of 10% for control

In [ ]:
control_conversions = np.random.binomial(1, 0.10, num_users_per_group)
control_clicks = np.random.randint(0, 15, num_users_per_group) # Clicks per user
control_page_views = control_clicks + np.random.randint(1, 5, num_users_per_group) # Page views > clicks
control_page_views = np.maximum(1, control_page_views) # At least 1 page view

Treatment Group (B) - let's assume a slight lift

In [ ]:
treatment_user_ids = [f'UserB_{20000+i}' for i in range(num_users_per_group)]
treatment_group_assignment = ['Treatment'] * num_users_per_group

Assume a target conversion rate of 11.5% for treatment (1.5% lift)

In [ ]:
treatment_conversions = np.random.binomial(1, 0.115, num_users_per_group)
treatment_clicks = np.random.randint(0, 17, num_users_per_group) # Slightly more clicks
treatment_page_views = treatment_clicks + np.random.randint(1, 5, num_users_per_group)
treatment_page_views = np.maximum(1, treatment_page_views)

 Combine data

In [ ]:
all_user_ids = control_user_ids + treatment_user_ids
all_groups = control_group_assignment + treatment_group_assignment
all_conversions = np.concatenate([control_conversions, treatment_conversions])
all_clicks = np.concatenate([control_clicks, treatment_clicks])
all_page_views = np.concatenate([control_page_views, treatment_page_views])

Assign random dates within the test period

In [ ]:
all_dates = [ (start_date + timedelta(days=np.random.randint(0, test_duration_days))).strftime('%Y-%m-%d')
              for _ in range(num_users_per_group * 2) ]
df_ab_test = pd.DataFrame({
    'UserID': all_user_ids,
    'Group': all_groups,
    'Date': all_dates,
    'PageViews': all_page_views,
    'Clicks': all_clicks,
    'Converted': all_conversions
})

Ensure clicks are not more than page views (though unlikely with current generation)

In [ ]:
df_ab_test['Clicks'] = df_ab_test.apply(lambda row: min(row['Clicks'], row['PageViews']), axis=1)

Save to CSV

In [ ]:
df_ab_test.to_csv('ab_test_results_mock_data.csv', index=False)
df_ab_test.sample(10, random_state=42)

,UserID,Group,Date,PageViews,Clicks,Converted
6252,UserB_21252,Treatment,2023-11-07,8,4,0
4684,UserA_14684,Control,2023-11-10,5,1,0
1731,UserA_11731,Control,2023-11-04,8,6,0
4742,UserA_14742,Control,2023-11-12,3,2,0
4521,UserA_14521,Control,2023-11-12,12,8,0
6340,UserB_21340,Treatment,2023-11-03,7,3,0
576,UserA_10576,Control,2023-11-08,4,0,0
5202,UserB_20202,Treatment,2023-11-05,15,14,1
6363,UserB_21363,Treatment,2023-11-12,16,14,1
439,UserA_10439,Control,2023-11-08,9,8,0


Basic check of conversion rates

In [ ]:
print("\nOverall Conversion Rates by Group:")
df_ab_test.groupby('Group')['Converted'].mean()


Overall Conversion Rates by Group:


,Converted
Group,
Control,0.0958
Treatment,0.1180


#Data cleaning

Check for missing data

In [ ]:
df_ab_test.isnull().sum()

,0
UserID,0
Group,0
Date,0
PageViews,0
Clicks,0
Converted,0


 Check for duplicate UserIDs in both groups

In [ ]:
user_group_counts = df_ab_test.groupby('UserID')['Group'].nunique()
user_group_counts[user_group_counts > 1]

,Group
UserID,


Summary statistics to detect outliers

In [ ]:
df_ab_test[['PageViews', 'Clicks', 'Converted']].describe()

,PageViews,Clicks,Converted
count,10000.000000,10000.000000,10000.000000
mean,10.004400,7.500500,0.106900
std,4.793963,4.665295,0.309002
min,1.000000,0.000000,0.000000
25%,6.000000,3.000000,0.000000
50%,10.000000,7.500000,0.000000
75%,14.000000,11.000000,0.000000
max,20.000000,16.000000,1.000000


 Check for users with PageViews < Clicks (illogical)

In [ ]:
illogical_rows = df_ab_test[df_ab_test['Clicks'] > df_ab_test['PageViews']]

Number of unique users vs total rows

In [ ]:
unique_users = df_ab_test['UserID'].nunique()
total_rows = len(df_ab_test)
display(name="Illogical Rows", dataframe=illogical_rows)

#Calculate Key Metrics:

Group by 'Group' and calculate conversion metrics

In [ ]:
df_ab_test_clean = df_ab_test[df_ab_test['Clicks'] <= df_ab_test['PageViews']]
conversion_metrics = df_ab_test_clean.groupby('Group').agg(
    Total_Users=('UserID', 'count'),
    Total_Conversions=('Converted', 'sum')
)

Calculate conversion rate

In [ ]:
conversion_metrics['Conversion_Rate'] = (
    conversion_metrics['Total_Conversions'] / conversion_metrics['Total_Users']
)
conversion_metrics

,Total_Users,Total_Conversions,Conversion_Rate
Group,,,
Control,5000,479,0.0958
Treatment,5000,590,0.1180


Calculate observed difference in conversion rates

In [ ]:
observed_difference = (
    conversion_metrics.loc['Treatment', 'Conversion_Rate'] -
    conversion_metrics.loc['Control', 'Conversion_Rate']
)
print(observed_difference)

0.022199999999999998


#Statistical Significance Testing Results

**Hypotheses**

Null Hypothesis (H₀): The conversion rates are the same for control and treatment groups.

Alternative Hypothesis (H₁): The conversion rates are different between the groups.



Extract data for Z-test

In [ ]:
conversions = conversion_metrics['Total_Conversions'].values
totals = conversion_metrics['Total_Users'].values

Perform Z-test for proportions

In [ ]:
z_stat, p_value = proportions_ztest(count=conversions, nobs=totals)
print(z_stat)

-3.59239498548479


In [ ]:
print(p_value)

0.000327652709954209


Calculate confidence interval for the difference in proportions

In [ ]:
diff = conversion_metrics.loc['Treatment', 'Conversion_Rate'] - conversion_metrics.loc['Control', 'Conversion_Rate']
se = np.sqrt(
    (conversion_metrics.loc['Control', 'Conversion_Rate'] * (1 - conversion_metrics.loc['Control', 'Conversion_Rate']) / totals[0]) +
    (conversion_metrics.loc['Treatment', 'Conversion_Rate'] * (1 - conversion_metrics.loc['Treatment', 'Conversion_Rate']) / totals[1])
)
margin_of_error = 1.96 * se
ci_diff = (diff - margin_of_error, diff + margin_of_error)
print(ci_diff)

(np.float64(0.010095564285965246), np.float64(0.03430443571403475))


#Practical Significance and Power Analysis

Calculate practical uplift

In [ ]:
conversion_control = conversion_metrics.loc['Control', 'Conversion_Rate']
conversion_treatment = conversion_metrics.loc['Treatment', 'Conversion_Rate']
uplift = (conversion_treatment - conversion_control) / conversion_control * 100  # in percent
print(uplift)

23.173277661795407


Power analysis: only relevant if the result was NOT significant, but we include this for completeness using statsmodels

In [ ]:
effect_size = sm.stats.proportion_effectsize(conversion_treatment, conversion_control)
analysis = NormalIndPower()
power = analysis.power(effect_size=effect_size, nobs1=totals[0], alpha=0.05, ratio=1.0, alternative='two-sided')

print("effect_size is ",effect_size)

effect_size is  0.07193882250897876


In [ ]:
print("Power is",power)

Power is 0.949182390735705
